In [3]:
var = [1, 4, 2, 5, 3]


In [4]:
for i in map(str, var):
    print(i, type(i))


1 <class 'str'>
4 <class 'str'>
2 <class 'str'>
5 <class 'str'>
3 <class 'str'>


In [5]:
var.append(0)
" ".join(map(str, var))


'1 4 2 5 3 0'

In [6]:
def encodage(ne, nj, j, x, y):
    return j * ne**2 + x * ne + y + 1


def decodage(k, ne):
    y = (k - 1) % ne
    x = ((k - 1 - y) // ne) % ne
    j = (k - 1 - y - x * ne) // (ne * ne)
    return j, x, y


print(encodage(3, 4, 1, 0, 0))
print(decodage(10, 3))


10
(1, 0, 0)


In [7]:
var = [1, 2, 3]


def au_moins_un(variables):
    clause = [str(v) for v in variables]
    clause.append("0")
    return [" ".join(clause)]


au_moins_un(var)


['1 2 3 0']

In [8]:
var = [1, 2, 3]


def au_plus_un(variables):
    clauses = []
    n = len(variables)
    for i in range(n):
        for j in range(i + 1, n):
            clauses.append(f"{-variables[i]} {-variables[j]} 0")
    return clauses


au_plus_un(var)


['-1 -2 0', '-1 -3 0', '-2 -3 0']

In [9]:
from itertools import combinations

l = [[-i, -j] for i, j in combinations(var, 2)]
print(l)
[" ".join(map(str, clause + [0])) for clause in l]


[[-1, -2], [-1, -3], [-2, -3]]


['-1 -2 0', '-1 -3 0', '-2 -3 0']

In [84]:
def encoderC1(ne, nj):
    clauses = []
    for j in range(nj):
        for x in range(ne):
            # Contrainte "au plus 1" pour chaque équipe x et chaque jour j
            domicile = [encodage(ne, nj, j, x, y) for y in range(ne) if y != x]
            exterieur = [encodage(ne, nj, j, y, x) for y in range(ne) if y != x]
            clauses.extend(au_plus_un(domicile + exterieur))
    return clauses


len(encoderC1(3, 4))


72

In [11]:
def encoderC2(ne, nj):
    clauses = []
    for x in range(ne):
        for y in range(ne):
            if x != y:
                list_match = [encodage(ne, nj, j, x, y) for j in range(nj)]
                clauses.extend(au_moins_un(list_match))
                clauses.extend(au_plus_un(list_match))
    return clauses


print(len(encoderC2(3, 4)))
encoderC2(3, 4)


42


['2 11 20 29 0',
 '-2 -11 0',
 '-2 -20 0',
 '-2 -29 0',
 '-11 -20 0',
 '-11 -29 0',
 '-20 -29 0',
 '3 12 21 30 0',
 '-3 -12 0',
 '-3 -21 0',
 '-3 -30 0',
 '-12 -21 0',
 '-12 -30 0',
 '-21 -30 0',
 '4 13 22 31 0',
 '-4 -13 0',
 '-4 -22 0',
 '-4 -31 0',
 '-13 -22 0',
 '-13 -31 0',
 '-22 -31 0',
 '6 15 24 33 0',
 '-6 -15 0',
 '-6 -24 0',
 '-6 -33 0',
 '-15 -24 0',
 '-15 -33 0',
 '-24 -33 0',
 '7 16 25 34 0',
 '-7 -16 0',
 '-7 -25 0',
 '-7 -34 0',
 '-16 -25 0',
 '-16 -34 0',
 '-25 -34 0',
 '8 17 26 35 0',
 '-8 -17 0',
 '-8 -26 0',
 '-8 -35 0',
 '-17 -26 0',
 '-17 -35 0',
 '-26 -35 0']

In [12]:
def encoder(ne, nj):
    contraintes = encoderC1(ne, nj) + encoderC2(ne, nj)
    with open("contraintes.cnf", "w") as f:
        # Ecrire l'en-tête du fichier DIMACS
        f.write("p cnf {} {}\n".format(ne * ne * nj - 1, len(contraintes)))
        # Ecrire les contraintes
        for contrainte in contraintes:
            f.write(contrainte + "\n")

In [81]:
def decoder(output_file, ne, nj, team_names_file=None):
    # Ouvrir le fichier contenant les noms d'équipe
    if team_names_file is not None:
        with open(team_names_file, "r") as f:
            team_names = [line.strip() for line in f]

    # Ouvrir le fichier de sortie de glucose
    with open(output_file, "r") as f:
        output_lines = f.readlines()

    # Extraire le modèle s'il existe
    model = output_lines[0].split()

    # Si aucun modèle n'a été trouvé, il n'y a pas de solution
    if "UNSAT" in model:
        return None

    # Cas SAT : initialisation du planning sous forme de dictionnaire
    planning = {j: {"Domicile" : "", "Extérieur": ""} for j in range(nj)}

    for s in model:
        s = int(s)
        if s > 0:
            j, x, y = decodage(s, ne)
            if team_names_file != None:
                x = team_names[x]
                y = team_names[y]
            planning[j]["Domicile"] = x
            planning[j]["Extérieur"] = y

    return planning

decoder("output.txt", 3, 10)


{0: {'Domicile': 1, 'Extérieur': 0},
 1: {'Domicile': 0, 'Extérieur': 1},
 2: {'Domicile': '', 'Extérieur': ''},
 3: {'Domicile': 0, 'Extérieur': 2},
 4: {'Domicile': '', 'Extérieur': ''},
 5: {'Domicile': '', 'Extérieur': ''},
 6: {'Domicile': 1, 'Extérieur': 2},
 7: {'Domicile': 2, 'Extérieur': 0},
 8: {'Domicile': '', 'Extérieur': ''},
 9: {'Domicile': 2, 'Extérieur': 1}}

In [82]:
planning = decoder("output.txt", 3, 10, "equipes.txt")
# print("{:<10} {:<15} {}".format('Jour','Domicile','Extérieur'))
# for jour, matchs in planning.items():
#     domicile = matchs["Domicile"]
#     exterieur = matchs["Extérieur"]
#     print(" {:<8} {:<15} {:<10}".format(jour + 1, domicile, exterieur))

In [54]:
column_lengths = {"Jour": max(len(str(row_name)) for row_name in planning.keys())}
for row in planning.values():
    for column, value in row.items():
        column_lengths[column] = max(column_lengths.get(column, 0), len(str(value)))

column_lengths

{'Jour': 1, 'Domicile': 19, 'Extérieur': 19}

In [53]:
 # Print the table header
headers =(column_lengths.keys())
print("  | " + " | ".join(f"{header:^{column_lengths[header]}}" for header in headers) + " |")

# Print the table separator
print("-+-".join("-" * column_lengths[header] for header in headers) + "-+")

# Print each row
for row_num, row in planning.items():
    print(f"{row_num + 1} | " + " | ".join(f"{str(value):^{column_lengths[column]}}" for column, value in row.items()) + " |")


  |   Jour   |      Domicile       |      Extérieur      |
---------+---------------------+---------------------+
1 |    Saint-Etienne    | Paris Saint-Germain |
2 | Paris Saint-Germain |    Saint-Etienne    |
3 | Paris Saint-Germain |       Monaco        |
4 |    Saint-Etienne    |       Monaco        |
5 |       Monaco        | Paris Saint-Germain |
6 |       Monaco        |    Saint-Etienne    |


In [83]:
def print_table(data):
    row_column_name = "Jour"
    # Determine the maximum length of each column
    column_lengths = {}
    for row in data.values():
        for column, value in row.items():
            column_lengths[column] = max(column_lengths.get(column, 0), len(str(value)))

    # Print the table header
    headers = sorted(data[0].keys())
    print(f"{row_column_name} | " + " | ".join(f"{header:^{column_lengths[header]}}" for header in headers) + " |")

    # Print the table separator
    print("-----+-" + "-+-".join("-" * column_lengths[header] for header in headers) + "-+")

    # Print each row
    for row_num, row in data.items():
        print(f"{str(row_num + 1):<{len(row_column_name)}} | " + " | ".join(f"{str(value):^{column_lengths[column]}}" for column, value in row.items()) + " |")

print_table(planning)

Jour |      Domicile       |      Extérieur      |
-----+---------------------+---------------------+
1    |    Saint-Etienne    | Paris Saint-Germain |
2    | Paris Saint-Germain |    Saint-Etienne    |
3    |                     |                     |
4    | Paris Saint-Germain |       Monaco        |
5    |                     |                     |
6    |                     |                     |
7    |    Saint-Etienne    |       Monaco        |
8    |       Monaco        | Paris Saint-Germain |
9    |                     |                     |
10   |       Monaco        |    Saint-Etienne    |


In [77]:
def print_table(data):
    # Determine the maximum length of each column
    column_lengths = {"Jour": 6}
    for row in data.values():
        for column, value in row.items():
            column_lengths[column] = max(column_lengths.get(column, 0), len(str(value)))

    # Print the table header
    headers = ["Jour"] + sorted(data[0].keys())
    print(" | ".join(f"{header:^{column_lengths[header]}}" for header in headers))

    # Print the table separator
    print("-+-".join("-" * column_lengths[header] for header in headers))

    # Print each row
    for row_name, row in data.items():
        row_values = [str(row_name)]
        row_values.extend(str(value) for value in row.values())
        print(" | ".join(f"{value:^{column_lengths[column]}}" for column, value in zip(headers, row_values)))

print_table(planning)

 Jour  |      Domicile       |      Extérieur     
-------+---------------------+--------------------
  0    |    Saint-Etienne    | Paris Saint-Germain
  1    | Paris Saint-Germain |    Saint-Etienne   
  2    | Paris Saint-Germain |       Monaco       
  3    |    Saint-Etienne    |       Monaco       
  4    |       Monaco        | Paris Saint-Germain
  5    |       Monaco        |    Saint-Etienne   


In [76]:
def print_table(planning):
    # Determine the maximum length of each column
    column_lengths = {"Jour": 4}
    for row in planning.values():
        for column, value in row.items():
            column_lengths[column] = max(column_lengths.get(column, 0), len(str(value)))

    # Print the table top border
    headers = ["Jour"] + sorted(planning[0].keys())
    print("+" + "+".join("-" * (column_lengths[column] + 2) for column in headers) + "+")

    # Print the table header
    print("| " + " | ".join(f"{header:^{column_lengths[header]}}" for header in headers) + " |")

    # Print the table header separator
    print("+" + "+".join("-" * (column_lengths[column] + 2) for column in headers) + "+")

    # Print each row
    for row_name, row in planning.items():
        row_values = [str(row_name)]
        row_values.extend(str(value) for value in row.values())
        print("| " + " | ".join(f"{value:^{column_lengths[column]}}" for column, value in zip(headers, row_values)) + " |")

    # Print the table bottom border
    print("+" + "+".join("-" * (column_lengths[column] + 2) for column in headers) + "+")

print_table(planning)

+------+---------------------+---------------------+
| Jour |      Domicile       |      Extérieur      |
+------+---------------------+---------------------+
|  0   |    Saint-Etienne    | Paris Saint-Germain |
|  1   | Paris Saint-Germain |    Saint-Etienne    |
|  2   | Paris Saint-Germain |       Monaco        |
|  3   |    Saint-Etienne    |       Monaco        |
|  4   |       Monaco        | Paris Saint-Germain |
|  5   |       Monaco        |    Saint-Etienne    |
+------+---------------------+---------------------+


In [67]:
def print_table(data):
    # Determine the maximum length of each column
    column_lengths = {"row_name": max(len(str(row_name)) for row_name in data.keys())}
    for row in data.values():
        for column, value in row.items():
            column_lengths[column] = max(column_lengths.get(column, 0), len(str(value)))

    # Print the table top border
    print("+" + "+".join("-" * (column_lengths[column] + 2) for column in ["row_name"] + sorted(data[0].keys())) + "+")

    # Print the table header
    headers = sorted(data[0].keys())
    print("| " + " | ".join(f"{header:^{column_lengths[header]}}" for header in ["row_name"] + headers) + " |")

    # Print the table header separator
    print("+" + "+".join("=" * (column_lengths[column] + 2) for column in ["row_name"] + headers) + "+")

    # Print each row
    for row_name, row in data.items():
        row_values = [str(row_name)]
        row_values.extend(str(value) for value in row.values())
        print("| " + " | ".join(f"{value:^{column_lengths[column]}}" for column, value in zip(["row_name"] + headers, row_values)) + " |")

        # Print the row separator
        print("+" + "+".join("-" * (column_lengths[column] + 2) for column in ["row_name"] + sorted(data[0].keys())) + "+")

print_table(planning)

+---+---------------------+---------------------+
| row_name |      Domicile       |      Extérieur      |
+===+=====================+=====================+
| 0 |    Saint-Etienne    | Paris Saint-Germain |
+---+---------------------+---------------------+
| 1 | Paris Saint-Germain |    Saint-Etienne    |
+---+---------------------+---------------------+
| 2 | Paris Saint-Germain |       Monaco        |
+---+---------------------+---------------------+
| 3 |    Saint-Etienne    |       Monaco        |
+---+---------------------+---------------------+
| 4 |       Monaco        | Paris Saint-Germain |
+---+---------------------+---------------------+
| 5 |       Monaco        |    Saint-Etienne    |
+---+---------------------+---------------------+


In [38]:
with open("salut.txt", "r") as f:
    output_lines = f.readlines()

    # Extraire le modèle s'il existe
"UNSAT" in output_lines[0].split() 

True

In [48]:
import subprocess
import os
def optimisation(ne, nj_min, nj_max):
    import multiprocessing
    def glucose():
        cmd = "glucose contraintes.cnf optimisation.txt"
        subprocess.run(cmd, shell=True)

    for j in range(nj_min, nj_max):
        encoder(ne, j)
        p = multiprocessing.Process(target=glucose)
        p.start()
        p.join(10)  # on attend 10 secondes
        if p.is_alive(): # si le processus est toujours vivant après 10 secondes
            print("timeout atteint... ")
            p.terminate() # on le termine
            continue # on passe à l'itération suivante
        if decoder("optimisation.txt", ne, j) is not None:
            return j

c
c This is glucose 4.2.1 --  based on MiniSAT (Many thanks to MiniSAT team)
c
c ========================================[ Problem Statistics ]===========================================
c |                                                                                                       |
c |  Number of variables:           143                                                                   |
c |  Number of clauses:            2772                                                                   |
c |  Parse time:                   0.00 s                                                                 |
c |                                                                                                       |
c | Preprocesing is fully done
c |  Simplification time:          0.00 s                                                                 |
c |                                                                                                       |
c ========================

In [16]:
os.getcwd()

'/home/aymeric/Documents/DAC/S2/IAMSI/tme04'

In [55]:
encoder(12, 20)

In [56]:
p = multiprocessing.Process(target = oui)
p.start()

c
c This is glucose 4.2.1 --  based on MiniSAT (Many thanks to MiniSAT team)
c
c ========================================[ Problem Statistics ]===========================================
c |                                                                                                       |
c |  Number of variables:          2879                                                                   |
c |  Number of clauses:           80652                                                                   |
c |  Parse time:                   0.01 s                                                                 |
c |                                                                                                       |
c | Preprocesing is fully done
c |  Eliminated clauses:           0.01 Mb                                                                |
c |  Simplification time:          0.10 s                                                                 |
c |                       